In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F

from helper import *
from dataset_class import *
from unet_class import *
from siamens import *

In [2]:
# Load images from folders A and B
folder_A = 'trainval/A'
folder_B = 'trainval/B'
folder_label = 'trainval/label'

In [25]:
device = ("cuda" if torch.cuda.is_available() else "cpu") # Use GPU or CPU for training

In [26]:
print(device)

cuda


In [5]:
images_A = load_images_from_folder(folder_A, is_gray=False)


In [6]:
images_B = load_images_from_folder(folder_B, is_gray=False)


In [7]:
labels = load_images_from_folder(folder_label, is_gray = False)

In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5, 0.5),
])

In [9]:
dataset = ChangeDetectionDataset(images_A, images_B, labels, transform=transform)

In [10]:
del images_A, images_B, labels

In [11]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

In [12]:
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [13]:
del dataset

In [14]:
batch_size = 16

In [15]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [16]:
del train_dataset,val_dataset

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F  # Added import for functional module
from torch.nn.functional import relu, sigmoid

class ChangeDetectionUNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Encoder
        self.e11 = nn.Conv2d(6, 64, kernel_size=3 , padding=1)
        self.e12 = nn.Conv2d(64, 64, kernel_size=3 , padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.e21 = nn.Conv2d(64, 128, kernel_size=3 , padding=1)
        self.e22 = nn.Conv2d(128, 128, kernel_size=3 , padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.e31 = nn.Conv2d(128, 256, kernel_size=3 , padding=1)
        self.e32 = nn.Conv2d(256, 256, kernel_size=3 , padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.e41 = nn.Conv2d(256, 512, kernel_size=3 , padding=1)
        self.e42 = nn.Conv2d(512, 512, kernel_size=3 , padding=1)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.e51 = nn.Conv2d(512, 1024, kernel_size=3 , padding=1)
        self.e52 = nn.Conv2d(1024, 1024, kernel_size=3 , padding=1)

        # Decoder
        self.upconv1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.d11 = nn.Conv2d(1024, 512, kernel_size=3 , padding=1)
        self.d12 = nn.Conv2d(512, 512, kernel_size=3 , padding=1)

        self.upconv2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.d21 = nn.Conv2d(512, 256, kernel_size=3 , padding=1)
        self.d22 = nn.Conv2d(256, 256, kernel_size=3 , padding=1)

        self.upconv3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.d31 = nn.Conv2d(256, 128, kernel_size=3 , padding=1)
        self.d32 = nn.Conv2d(128, 128, kernel_size=3 , padding=1)

        self.upconv4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.d41 = nn.Conv2d(128, 64, kernel_size=3 , padding=1)
        self.d42 = nn.Conv2d(64, 64, kernel_size=3 , padding=1)

        # Output layer
        self.outconv = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x1, x2):
        
        # Concatenate input images along the channel dimension
        x = torch.cat([x1, x2], dim=1)

        # Encoder
        xe11 = torch.relu(self.e11(x))
        xe12 = torch.relu(self.e12(xe11))
        xp1 = self.pool1(xe12)


        xe21 = torch.relu(self.e21(xp1))
        xe22 = torch.relu(self.e22(xe21))
        xp2 = self.pool2(xe22)


        xe31 = torch.relu(self.e31(xp2))
        xe32 = torch.relu(self.e32(xe31))
        xp3 = self.pool3(xe32)


        xe41 = torch.relu(self.e41(xp3))
        xe42 = torch.relu(self.e42(xe41))
        xp4 = self.pool4(xe42)


        xe51 = torch.relu(self.e51(xp4))
        xe52 = torch.relu(self.e52(xe51))


        # Decoder
        xu1 = self.upconv1(xe52)
        # xe42_resized = F.interpolate(xe42, size=xu1.size()[2:], mode='bilinear', align_corners=True)
        xu11 = torch.cat([xe42, xu1], dim=1)
        xd11 = torch.relu(self.d11(xu11))
        xd12 = torch.relu(self.d12(xd11))

        xu2 = self.upconv2(xd12)
        # xe32_resized = F.interpolate(xe32, size=xu2.size()[2:], mode='bilinear', align_corners=True)
        xu22 = torch.cat([xe32, xu2], dim=1)
        xd21 = torch.relu(self.d21(xu22))
        xd22 = torch.relu(self.d22(xd21))

        xu3 = self.upconv3(xd22)
        # xe22_resized = F.interpolate(xe22, size=xu3.size()[2:], mode='bilinear', align_corners=True)
        xu33 = torch.cat([xe22, xu3], dim=1)
        xd31 = torch.relu(self.d31(xu33))
        xd32 = torch.relu(self.d32(xd31))

        xu4 = self.upconv4(xd32)
        # xe12_resized = F.interpolate(xe12, size=xu4.size()[2:], mode='bilinear', align_corners=True)
        xu44 = torch.cat([xe12, xu4], dim=1)
        xd41 = torch.relu(self.d41(xu44))
        xd42 = torch.relu(self.d42(xd41))

        # Output layer with sigmoid activation
        out = torch.sigmoid(self.outconv(xd42))

        
        return out


In [27]:
model = ChangeDetectionUNet()
model = model.to(device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [21]:
criterion = nn.BCEWithLogitsLoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0005)

In [20]:
# Training loop
num_epochs = 3
best_losses = 100000000

for epoch in range(num_epochs):
    
    total_true = []
    total_pred = []
    
    model.train()
    total_train_loss = 0
    total_intersection = 0
    total_union = 0
    
    for batch_idx, (data_A, data_B, target) in enumerate(train_loader):
        print("Batch index train:",batch_idx)
        optimizer.zero_grad()
        data_A = data_A.float().cuda()
        data_B = data_B.float().cuda()
        target = target.float().cuda()
        output = model(data_A, data_B)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step() 
        total_train_loss += loss.item()



    # Validation
    model.eval()
    total_val_loss = 0
    total_val_intersection = 0
    total_val_union = 0
    
    with torch.no_grad():
        for batch_idx, (data_A, data_B, target) in enumerate(val_loader):
            print("Batch index test:",batch_idx)
            data_A = data_A.float().cuda()
            data_B = data_B.float().cuda()
            target = target.float().cuda()
            output = model(data_A, data_B)
            val_loss = criterion(output, target)
            total_val_loss += val_loss.item()
            
            out_binary = torch.where(output > 0.5, torch.tensor(1.0, requires_grad= True), torch.tensor(0.0, requires_grad= True))

            total_pred.append(out_binary)
            total_true.append(target)

    if(total_val_loss < best_losses):
        best_losses = total_val_loss
        torch.save(model.state_dict(), 'model_9.pickle')
    
   
        
    print(f'Epoch {epoch+1}, Train Loss: {total_train_loss/len(train_loader)}'
          f'Validation Loss: {total_val_loss/len(val_loader)}')


Batch index train: 0


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [21]:
# del model, criterion, optimizer

In [22]:
torch.save(model.state_dict(), 'model8.pickle')

In [23]:
temp = total_pred

In [24]:
total_pred = temp

In [25]:
temp2 = total_true

In [26]:
model.eval()

ChangeDetectionUNet(
  (e11): Conv2d(6, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (e12): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (e21): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (e22): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (e31): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (e32): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (e41): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (e42): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (e51): Conv2d(512, 1024

In [27]:
for i in range(len(total_pred)):
    total_pred[i] = total_pred[i].to('cpu')
    total_true[i] = total_true[i].to('cpu')

In [28]:
print(temp[7])

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0.

In [29]:
def calculate_jaccard_index(pred_mask, true_mask):
    intersection = np.logical_and(pred_mask, true_mask).sum()
    union = np.logical_or(pred_mask, true_mask).sum()
    jaccard_index = intersection / union
    return jaccard_index

In [30]:
for i in range(len(total_pred)):
    print(calculate_jaccard_index(total_pred[i], total_true[i]))

tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)

In [31]:
# for i in range(len(total_pred)):
#     print(torch.sum(total_pred[i]))

In [32]:
# model.eval()

In [33]:

# def calculate_jaccard_index(pred_mask, true_mask):
#     intersection = np.logical_and(pred_mask, true_mask).sum()
#     union = np.logical_or(pred_mask, true_mask).sum()
#     jaccard_index = intersection / union
#     return jaccard_index

In [34]:
# # Validation
# model.eval()
# intersection = 0
# union = 0

# with torch.no_grad():
#     for batch_idx, (data_A, data_B, target) in enumerate(val_loader):
#         if (batch_idx == 1):
#             data_A = data_A.float().to(device)
#             data_B = data_B.float().to(device)
#             target = target.float().to(device)

#             output = model(data_A, data_B)
#             output = output > 0.3
#             print(calculate_jaccard_index(output.to('cpu'), target.to('cpu')))
#             # predicted_masks = (output > 0.5).float()  # Threshold predicted probabilities
